<a href="https://colab.research.google.com/github/MarcoBessiNeo4j/Demos/blob/main/ShortDemo_Merchant_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Demo GDS+VectorIndex+OpenAI**

In [ ]:
pip install graphdatascience openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
  Created wheel for neo4j: filename=neo4j-5.19.0-py3-none-any.whl size=280741 sha256=42ff885edb3ff78b66d3528b2e265c72ae00f8e4882470ae6323c07ac5a82c99
  Stored in directory: /root/.cache/pip/wheels/30/db/9b/2cfde1fa33145219c0322f299b604daf5aba2ed443a7ed5f07
Successfully built neo4j


In [ ]:
from graphdatascience import GraphDataScience
import openai
import pandas as pd

*Setup variabili*

In [ ]:
# get your Neo4j instance on https://neo4j.com/sandbox/
from pprint import pprint
from google.colab import userdata

bolt = userdata.get('NEO4J_URL_SANDBOX_MERCHANT')
user = userdata.get('NEO4J_USR')
password = userdata.get('NEO4J_PWD_SANDBOX_MERCHANT')
auth = (user, password)
openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
gds = GraphDataScience(bolt, auth=auth, aura_ds=False)
print(gds.version())

2.6.3


*Caricamento dataset in Neo4j*

In [ ]:
transaction_df = pd.DataFrame([
    {'name': 'Anna', 'merchant':'Amazon', 'amount': 100},
    {'name': 'Anna', 'merchant':'Dustin', 'amount': 5499},
    {'name': 'Anna', 'merchant':'eBay', 'amount': 220},
    {'name': 'Jonas', 'merchant':'Amazon', 'amount': 220},
    {'name': 'Jonas', 'merchant':'Dustin', 'amount': 399},
    {'name': 'Jonas', 'merchant':'eBay', 'amount': 1499},
    {'name': 'Jonas', 'merchant':'Bikes.de', 'amount': 2000},
    {'name': 'Kristof', 'merchant':'Amazon', 'amount': 423},
    {'name': 'Kristof', 'merchant':'Dustin', 'amount': 530},
    {'name': 'Kristof', 'merchant':'Hello Fresh', 'amount': 1050},
    {'name': 'Kristof', 'merchant':'Steam', 'amount': 230},
    {'name': 'Kristof', 'merchant':'Activision', 'amount': 783},
    {'name': 'Jonas', 'merchant':'Bikes.de', 'amount': 22000},
    {'name': 'Håkan', 'merchant':'Hello Fresh', 'amount': 2100},
    {'name': 'Håkan', 'merchant':'Steam', 'amount': 230},
    {'name': 'Håkan', 'merchant':'Activision', 'amount': 783},

], columns = ['name', 'merchant', 'amount'])
transaction_df.head(15)

,name,merchant,amount
0,Anna,Amazon,100
1,Anna,Dustin,5499
2,Anna,eBay,220
3,Jonas,Amazon,220
4,Jonas,Dustin,399
5,Jonas,eBay,1499
6,Jonas,Bikes.de,2000
7,Kristof,Amazon,423
8,Kristof,Dustin,530
9,Kristof,Hello Fresh,1050


In [ ]:
transaction_df.head(15)

,name,merchant,amount
0,Anna,Amazon,100
1,Anna,Dustin,5499
2,Anna,eBay,220
3,Jonas,Amazon,220
4,Jonas,Dustin,399
5,Jonas,eBay,1499
6,Jonas,Bikes.de,2000
7,Kristof,Amazon,423
8,Kristof,Dustin,530
9,Kristof,Hello Fresh,1050


In [ ]:
gds.run_cypher(
    """
    unwind $transactions as transaction
    merge (p:Person{name: transaction['name']})
    merge (m:Merchant{name: transaction['merchant']})
    merge (p)-[tx:TRANSACTED_WITH]->(m)
       set tx.amount = transaction['amount']
    """,
    params = { 'transactions': transaction_df.to_dict(orient='records') }
)

""


*Feature engineering sul dataset in Neo4j con GDS*

In [ ]:
G, res = gds.graph.project(
    "shopping",                                     #  Graph name
    ["Person", "Merchant"],                         #  Node projection
    {"TRANSACTED_WITH": {                           #  Relationship projection
        "properties": "amount",
        "orientation": "REVERSE"}
    }
)

In [ ]:
write_results = gds.nodeSimilarity.write(
    G,
    writeRelationshipType= 'SIMILAR_CUSTOMERS' ,
    writeProperty= 'SIM_SCORE',
    relationshipWeightProperty= 'amount'
)

gds.run_cypher("""
    MATCH (m:Merchant)-[r:SIMILAR_CUSTOMERS]->(n:Merchant)
    WHERE (n)-[:SIMILAR_CUSTOMERS]->(m) AND id(m)<id(n)
    DELETE r
""")


write_results

preProcessingMillis                                                       0
computeMillis                                                            11
writeMillis                                                              87
postProcessingMillis                                                      0
nodesCompared                                                             7
relationshipsWritten                                                     30
similarityDistribution    {'min': 0.009767770767211914, 'p5': 0.00976777...
configuration             {'writeProperty': 'SIM_SCORE', 'writeRelations...
Name: 0, dtype: object

*OpenAI - Creazione descrizioni ed embeddings*

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai.api_key)

In [ ]:
def get_merchant_desc(merchant):
    completion = client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Provide a description of the merchant " + merchant + ". Use a maximum of 300 characters"}])
    return completion.choices[0].message.content

In [ ]:
merchant_df = gds.run_cypher("""
  match (m:Merchant)
  return distinct m.name as merchant
""")
merchant_df.head(10)

,merchant
0,Amazon
1,Dustin
2,eBay
3,Bikes.de
4,Hello Fresh
5,Steam
6,Activision


In [ ]:
merchant_df['description'] = merchant_df['merchant'].apply(get_merchant_desc)
merchant_df.head(10)

,merchant,description
0,Amazon,Amazon is a multinational technology giant fou...
1,Dustin,"Dustin is a young, ambitious merchant known fo..."
2,eBay,eBay is an online marketplace that connects bu...
3,Bikes.de,Bikes.de is a one-stop online shop for all you...
4,Hello Fresh,Hello Fresh is a meal kit delivery service tha...
5,Steam,Steam is a digital distribution platform devel...
6,Activision,Activision is a video game publisher known for...


In [ ]:
gds.run_cypher("""
  unwind $merchant_des as merchant_des
  MATCH  (m:Merchant {name: merchant_des['merchant']})
  set m.description=merchant_des['description']""",
  params = { 'merchant_des': merchant_df.to_dict(orient='records') }
)

""


In [ ]:
# Create a named node vector index for the specified label and property with the given vector dimensionality using either the EUCLIDEAN or COSINE similarity function.
gds.run_cypher("""CALL db.index.vector.createNodeIndex('description-embeddings', 'Merchant', 'embedding', 1536, 'cosine')""")

""


In [ ]:
# apoc.ml.openai.embedding Can take a list of text strings, and will return one row per string, with the embedding data as a 1536 element vector
# db.create.setVectorProperty Set a vector property on a given node in a more space efficient representation than Cypher’s SET.
gds.run_cypher("""
  MATCH (m:Merchant)
  WITH collect(m) as nodes
  UNWIND nodes as n
  CALL apoc.ml.openai.embedding([n.description], $apiKey, {}) yield embedding
  WITH n, embedding
  MATCH (n)
  CALL db.create.setVectorProperty(n, 'embedding', embedding)
  YIELD node
  RETURN COUNT(node)""",
  params = {'apiKey': openai.api_key}
)

,COUNT(node)
0,7


*OpenAI - Embedding della domanda con OpenAI e utilizzo del VectorIndex Neo4j per cercare la risposta corretta (con grafo di spiegazione)*

In [ ]:
search_phrase = input("Search phrase:")
#where can i find a bike?

# You can query a vector index using the procedure db.index.vector.queryNodes
gds.run_cypher("""
  CALL apoc.ml.openai.embedding([$search_phrase], $apiKey, {})
  YIELD embedding
  CALL db.index.vector.queryNodes('description-embeddings', 3, embedding)
  YIELD node, score
  MATCH (node)<-[:TRANSACTED_WITH]-(p:Person)
  RETURN node.name as merchant, node.description as merchantDescription, collect(p.name) as customers, score""",
  params = {'apiKey': openai.api_key, 'search_phrase': search_phrase}
)

Search phrase:where can i find a bike?


,merchant,merchantDescription,customers,score
0,Bikes.de,Bikes.de is a one-stop online shop for all you...,[Jonas],0.919685
1,eBay,eBay is an online marketplace that connects bu...,"[Jonas, Anna]",0.871613
2,Dustin,"Dustin is a young, ambitious merchant known fo...","[Kristof, Jonas, Anna]",0.856236


--------------------------------------------------------------------------------
Cancella per lasciare l'ambiente pulito.

In [ ]:
G.drop()
gds.run_cypher("""
  MATCH (n)
  DETACH DELETE n"""
)


""


DROP INDEX description-embeddings

In [ ]:
gds.run_cypher("""DROP INDEX `description-embeddings`""")

""
